In [13]:
import pandas as pd
import jsonpickle
import numpy as np
import matplotlib.pyplot as plt
import os, os.path
import dateutil.parser
from datetime import datetime

In [2]:
# FILE_AS_ROOT = False
SELF_PATH = os.getcwd()#os.path.dirname(os.path.abspath(__file__))
PATH_TO_ROOT = os.path.join(SELF_PATH,'../../../../../../')
PATH_TO_JSON_STATE = os.path.join(PATH_TO_ROOT,'tagging_data_personal.json')

In [3]:
def get_pickle(path_to_file):
    with open(path_to_file, 'r') as f:
        return jsonpickle.decode(f.read())

In [4]:
def how_many_tagged(pickle_file):
    
    done_tagging_count=len(pickle_file.finished_tagged_queue)
    tagged_but_not_done_count=0
    
    for image in pickle_file.pending_images_queue:
        if len(image.get_taggers()) > 0:
            tagged_but_not_done_count+=1
    
    return pd.DataFrame([{ 
        'not_done':tagged_but_not_done_count,
        'done':done_tagging_count,
        'tagged_ratio': tagged_but_not_done_count/(tagged_but_not_done_count+done_tagging_count)
     }])

In [5]:
pickle = get_pickle(PATH_TO_JSON_STATE)

In [6]:
how_many_tagged(pickle)

,not_done,done,tagged_ratio
0,117,248,0.320548


In [7]:
len(pickle.finished_tagged_queue)

248

In [8]:
print(pickle.pending_images_queue[-1].get_taggers())

{'auth0|5dcc69aee034f70d29923ce0'}


In [49]:
for image in pickle.finished_tagged_queue:
    times_stop = image.stats_tagging_stop
    keys = list(times_stop.keys())
    last_tagger = keys[-1]
    last_time = times_stop.get(last_tagger)
    print(datetime.fromtimestamp(last_time))

2019-11-11 15:17:08.593732
2019-11-11 15:17:59.522244
2019-11-11 15:18:17.454338
2019-11-11 15:18:40.977941
2019-11-11 15:19:44.631863
2019-11-11 15:19:55.251184
2019-11-11 15:20:07.482876
2019-11-11 15:20:17.263266
2019-11-11 15:21:14.809763
2019-11-11 15:23:08.146001
2019-11-11 15:23:21.354913
2019-11-11 15:23:32.832438
2019-11-11 15:24:22.148337
2019-11-11 15:25:25.229745
2019-11-11 15:25:35.188624
2019-11-11 15:26:00.618709
2019-11-11 15:26:11.831020
2019-11-11 15:26:20.891829
2019-11-11 15:29:19.281509
2019-11-11 15:33:38.356584
2019-11-11 15:33:55.526010
2019-11-11 15:35:33.332212
2019-11-11 15:35:43.372162
2019-11-11 15:35:53.355715
2019-11-11 15:36:18.452969
2019-11-11 15:36:28.318193
2019-11-11 16:03:57.141283
2019-11-12 01:32:38.789872
2019-11-12 01:33:03.153718
2019-11-12 01:33:28.828335
2019-11-12 01:35:01.769929
2019-11-12 01:35:17.975777
2019-11-12 01:35:38.402000
2019-11-12 01:35:49.207301
2019-11-12 01:36:35.075569
2019-11-12 01:38:26.573826
2019-11-12 01:38:51.531617
2